In [4]:
import os
import hashlib
from blake3 import blake3
from Crypto.Hash import keccak
from PIL import Image


ModuleNotFoundError: No module named 'Crypto'

In [ ]:
!pip install pycryptodome


In [ ]:
def calculate_hashes(image_path):
    """Calculate SHA-256, BLAKE3, and SHA3-256 (Keccak) hashes for an image file."""
    try:
        # Open and convert the image
        with Image.open(image_path) as img:
            # Convert to standard format to ensure consistent hashing
            img = img.convert('RGB')
            
            # Save image to bytes for hashing
            img_bytes = io.BytesIO()
            img.save(img_bytes, format='PNG')
            data = img_bytes.getvalue()
            
            # Calculate SHA-256 hash
            sha256_hash = hashlib.sha256(data).hexdigest()
            
            # Calculate BLAKE3 hash
            blake3_hash = blake3.blake3(data).hexdigest()
            
            # Calculate SHA3-256 (Keccak variant) hash using hashlib
            # Note: hashlib.sha3_256 implements Keccak finalized as SHA-3
            sha3_hash = hashlib.sha3_256(data).hexdigest()
            
            return {
                'file': os.path.basename(image_path),
                'sha256': sha256_hash,
                'blake3': blake3_hash,
                'sha3_256': sha3_hash
            }
    except Exception as e:
        return {
            'file': os.path.basename(image_path),
            'error': str(e)
        }


In [ ]:
def process_directory(directory_path):
    """Process all image files in a directory."""
    results = []
    
    # Accepted image file extensions
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif']
    
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path) and os.path.splitext(filename)[1].lower() in image_extensions:
            results.append(calculate_hashes(file_path))
    
    return results

In [ ]:
def main():
    parser = argparse.ArgumentParser(description='Calculate cryptographic hashes of biometric images.')
    group = parser.add_mutually_exclusive_group(required=True)
    group.add_argument('-f', '--file', help='Path to a single image file')
    group.add_argument('-d', '--directory', help='Path to a directory containing image files')
    parser.add_argument('-o', '--output', help='Output results to a text file')
    
    args = parser.parse_args()
    
    if args.file:
        results = [calculate_hashes(args.file)]
    else:
        results = process_directory(args.directory)
    
    # Display and/or save results
    output_text = ""
    for result in results:
        if 'error' in result:
            output_line = f"Error processing {result['file']}: {result['error']}"
            print(output_line)
            output_text += output_line + "\n"
        else:
            output_line = f"\nFile: {result['file']}"
            print(output_line)
            output_text += output_line + "\n"
            
            output_line = f"SHA-256:    {result['sha256']}"
            print(output_line)
            output_text += output_line + "\n"
            
            output_line = f"BLAKE3:     {result['blake3']}"
            print(output_line)
            output_text += output_line + "\n"
            
            output_line = f"SHA3-256:   {result['sha3_256']}"
            print(output_line)
            output_text += output_line + "\n"
    
    if args.output:
        with open(args.output, 'w') as f:
            f.write(output_text)
        print(f"\nResults saved to {args.output}")



In [ ]:
if __name__ == "__main__":
    main()